In [ ]:
## Use Bigquery to automatically process GWAS with sumstats.

import pandas as pd
from sqlalchemy import text
from sqlalchemy import create_engine

from google.cloud import bigquery

client = bigquery.Client()

# # Perform a query.
# QUERY = (
#     'WITH
#   SNP_info AS (
#   SELECT
#     CONCAT(CAST(chrom AS string), CAST(pos AS string), CAST(ref AS string), CAST(alt AS string)) AS identifier,
#     ref,
#     alt,
#     n_total,
#     pval,
#     eaf
#   FROM
#     `open-targets-genetics-dev.Jack_sumstats.GCST006867` )
# SELECT
#   *
# FROM
#   SNP_info
# JOIN (
#   SELECT
#     CONCAT(CAST(chr_id AS string), CAST(position AS string), CAST(ref_allele AS string), CAST(alt_allele AS string)) AS identifier,
#     rs_id
#   FROM
#     `open-targets-genetics.210608.variants` ) variants
# ON
#   SNP_info.identifier = variants.identifier')

# Perform a query.
QUERY = (
    'SELECT name FROM `bigquery-public-data.usa_names.usa_1910_2013` '
    'WHERE state = "TX" '
    'LIMIT 100')
    
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

for row in rows:
    print(row.name)


In [8]:
def implicit():
    from google.cloud import storage

    # If you don't specify credentials when constructing the client, the
    # client library will look for credentials in the environment.
    storage_client = storage.Client.from_service_account_json("/Users/xg1/Downloads/open-targets-genetics-eaa50bdaa2f5.json")

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)


In [ ]:
implicit()

In [14]:
Latest_study_table=pd.read_parquet("/Users/xg1/Downloads/v2d_210922_studies.parquet")
Latest_sumstats=Latest_study_table.loc[Latest_study_table["has_sumstats"] == True]
Latest_sumstats

,study_id,pmid,pub_date,pub_journal,pub_title,pub_author,trait_reported,trait_efos,ancestry_initial,ancestry_replication,n_initial,n_replication,n_cases,trait_category,num_assoc_loci,has_sumstats
0,FINNGEN_R5_AB1_AMOEBIASIS,,2021-5-11,,,FINNGEN_R5,Amoebiasis,[EFO_0007144],[European=197385],[],197385,0.0,87.0,Disease,0,True
1,FINNGEN_R5_AB1_ANOGENITAL_HERPES_SIMPLEX,,2021-5-11,,,FINNGEN_R5,Anogenital herpesviral [herpes simplex] infection,[EFO_0007282],[European=213843],[],213843,0.0,891.0,Disease of genitourinary system,0,True
2,FINNGEN_R5_AB1_ARTHROPOD,,2021-5-11,,,FINNGEN_R5,Arthropod-borne viral fevers and viral haemorr...,"[EFO_0000763, MONDO_0018087]",[European=218792],[],218792,0.0,1450.0,Disease,0,True
3,FINNGEN_R5_AB1_ASPERGILLOSIS,,2021-5-11,,,FINNGEN_R5,Aspergillosis,[EFO_0007157],[European=214903],[],214903,0.0,87.0,Disease,0,True
4,FINNGEN_R5_AB1_ATYPICAL_CNS_VIRUS,,2021-5-11,,,FINNGEN_R5,Atypical virus infections of central nervous s...,[MONDO_0024318],[European=217783],[],217783,0.0,146.0,Uncategorised,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36219,SAIGE_988,,2018-10-24,,,UKB SAIGE,Toxic effect of noxious substances eaten as food,[EFO_0008546],[European=408481],[],408481,0.0,114.0,Disease,0,True
36220,SAIGE_989,,2018-10-24,,,UKB SAIGE,"Toxic effect of other substances, chiefly nonm...",[EFO_0008546],[European=408425],[],408425,0.0,58.0,Disease,0,True
36221,SAIGE_990,,2018-10-24,,,UKB SAIGE,Effects radiation nos,[EFO_0009658],[European=406425],[],406425,0.0,3130.0,Disease,0,True
36222,SAIGE_994,,2018-10-24,,,UKB SAIGE,Sepsis and sirs,[HP_0100806],[European=408961],[],408961,0.0,2811.0,Uncategorised,0,True


In [17]:
import os
list_sumstats="bq.py --help"
os.system(list_sumstats)

32512

In [13]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# TODO(developer): Set table_id to the ID of the table to create.
table_id = "open-targets-genetics-dev.Jack_sumstats."+"your_table_name"

# job_config = bigquery.LoadJobConfig(
#     schema=[
#         bigquery.SchemaField("name", "STRING"),
#         bigquery.SchemaField("post_abbr", "STRING"),
#     ],
#     skip_leading_rows=1,
#     # The source format defaults to CSV, so the line below is optional.
#     source_format=bigquery.SourceFormat.CSV,
# )

job_config = bigquery.LoadJobConfig(source_format=bigquery.SourceFormat.PARQUET,)

uri = "gs://genetics-portal-dev-sumstats/unfiltered/gwas/GCST000568.parquet/*.parquet"

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = client.get_table(table_id)  # Make an API request.
print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 2445642 rows.


In [17]:
 # Make an API request.




In [18]:
query_job.result()

In [5]:
from google.cloud import bigquery
client = bigquery.Client()

Input_study="GCST000568"



## LOAD Job: load from GCS to BQ (table_id)
    # Would it be possible to make this table temporary? Or delete itself automatically after 1 week?

Input_sumstats_path="gs://genetics-portal-dev-sumstats/unfiltered/gwas"
Input_study_URI=Input_sumstats_path+"/"+Input_study+".parquet/*.parquet"

table_id = "open-targets-genetics-dev.Jack_sumstats."+Input_study

load_job_config = bigquery.LoadJobConfig(source_format=bigquery.SourceFormat.PARQUET,)
load_job = client.load_table_from_uri(
    Input_study_URI, table_id, job_config=load_job_config
)  # Make an API request.
load_job.result()  # Waits for the job to complete.

destination_table = client.get_table(table_id)  # Make an API request.
print("Loaded {} rows.".format(destination_table.num_rows))

# Query Job

client = bigquery.Client()
rsID_table="open-targets-genetics-dev.rsID_sumstats."+Input_study

query_job_config = bigquery.QueryJobConfig(destination=rsID_table)
query = """
    WITH SNP_info AS (
    SELECT
        CONCAT(CAST(chrom AS string), CAST(pos AS string), CAST(ref AS string), CAST(alt AS string)) AS identifier,
        ref,
        alt,
        n_total,
        pval,
        eaf,
        beta
    FROM
        `{0}` )
    SELECT
    rs_id AS RSID, ref AS A1, alt AS A2, n_total AS N, pval AS P, eaf AS EAF, beta AS BETA
    FROM
    SNP_info
    JOIN (
    SELECT
        CONCAT(CAST(chr_id AS string), CAST(position AS string), CAST(ref_allele AS string), CAST(alt_allele AS string)) AS identifier,
        rs_id
    FROM
        `open-targets-genetics.210608.variants` ) variants
    USING(identifier)
""".format(table_id)

query_job = client.query(query, job_config=query_job_config)
query_job.result()

rsID_GCS_bucket="gs://genetics-portal-dev-analysis/xg1/rsid_sumstats"
rsID_GCS_URI=rsID_GCS_bucket+"/{0}.txt".format(Input_study)

extract_job_config = bigquery.ExtractJobConfig() 
extract_job_config.field_delimiter = '\t'
extract_job_config.compression='GZIP'

extract_job = client.extract_table(
    rsID_table,
    rsID_GCS_URI,
    # Location must match that of the source table.
    location="EU",
    job_config=extract_job_config
)  # API request
extract_job.result()  # Waits for job to complete.

print(
    "Exported {} to {}".format(rsID_table, rsID_GCS_URI)
)


#print("The query data:")
#for row in query_job:
    # Row values can be accessed by field name or index.
    #print("name={}, count={}".format(row[0], row["total_people"]))


Loaded 12228210 rows.
Exported open-targets-genetics-dev.rsID_sumstats.GCST000568 to gs://genetics-portal-dev-analysis/xg1/rsid_sumstats/GCST000568


In [7]:
from google.cloud import bigquery
client = bigquery.Client()

#Input_study="GCST000568"
Input_study=["GCST005413", "GCST004131", "GCST005569", "GCST002245"]
## LOAD Job: load from GCS to BQ (table_id)
    # Would it be possible to make this table temporary? Or delete itself automatically after 1 week?

for study in Input_study:
    rsID_table="open-targets-genetics-dev.rsID_sumstats."+study

    rsID_GCS_bucket="gs://genetics-portal-dev-analysis/xg1/rsid_sumstats"
    rsID_GCS_URI=rsID_GCS_bucket+"/{0}.txt.gz".format(study)

    extract_job_config = bigquery.ExtractJobConfig() 
    extract_job_config.field_delimiter = '\t'
    extract_job_config.compression='GZIP'

    extract_job = client.extract_table(
        rsID_table,
        rsID_GCS_URI,
        # Location must match that of the source table.
        location="EU",
        job_config=extract_job_config
    )  # API request
    extract_job.result()  # Waits for job to complete.

    print(
        "Exported {} to {}".format(rsID_table, rsID_GCS_URI)
    )


Exported open-targets-genetics-dev.rsID_sumstats.GCST005413 to gs://genetics-portal-dev-analysis/xg1/rsid_sumstats/GCST005413.txt.gz
Exported open-targets-genetics-dev.rsID_sumstats.GCST004131 to gs://genetics-portal-dev-analysis/xg1/rsid_sumstats/GCST004131.txt.gz
Exported open-targets-genetics-dev.rsID_sumstats.GCST005569 to gs://genetics-portal-dev-analysis/xg1/rsid_sumstats/GCST005569.txt.gz
Exported open-targets-genetics-dev.rsID_sumstats.GCST002245 to gs://genetics-portal-dev-analysis/xg1/rsid_sumstats/GCST002245.txt.gz


In [15]:
from google.cloud import storage

ImportError: cannot import name 'storage' from 'google.cloud' (unknown location)

In [13]:
from google.cloud import bigquery

client = bigquery.Client()

# Perform a query.
QUERY = (
    'SELECT name FROM `bigquery-public-data.usa_names.usa_1910_2013` '
    'WHERE state = "TX" '
    'LIMIT 100')
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

for row in rows:
    print(row.name)

Frances
Alice
Beatrice
Ella
Gertrude
Josephine
Lula
Blanche
Marjorie
Christine
Margarita
Alta
Sara
Naomi
Sofia
Andrea
Delfina
Dominga
Elnora
Adele
Rafaela
Dixie
Luisa
Bess
Ernestine
Lorine
Patsy
Cecelia
Enriqueta
Lucinda
Alyce
Oneta
Apolonia
Gloria
Rhoda
Buna
Maye
Pansy
Gladys
Maria
Helen
Velma
Clara
Anna
Evelyn
Rosa
Mae
Bernice
Stella
Vivian
Mable
Maggie
Betty
Guadalupe
Lorene
Susie
Sadie
Erma
Verna
Amelia
Estelle
Nell
Margie
Celia
Lessie
Eloise
Millie
Olga
Letha
Rachel
Luz
Gussie
Sylvia
Aline
Barbara
Iris
Leila
Mozelle
Simona
Catalina
Hester
Willia
Allene
Avis
George
Isabella
Polly
Syble
Teodora
Lennie
Ricarda
Valerie
Frances
Gertrude
Sarah
Rosa
Lucy
Nora
Nettie
Odessa


In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

query = """
    SELECT name, SUM(number) as total_people
    FROM `bigquery-public-data.usa_names.usa_1910_2013`
    WHERE state = 'TX'
    GROUP BY name, state
    ORDER BY total_people DESC
    LIMIT 20
"""
query_job = client.query(query)  # Make an API request.

print("The query data:")
for row in query_job:
    # Row values can be accessed by field name or index.
    print("name={}, count={}".format(row[0], row["total_people"]))

In [35]:
import pandas as pd

Full_baseline=pd.read_csv("/Users/xg1/Downloads/ldsc/baselineLD/baselineLD.22.annot.gz", sep='\t')
list(Full_baseline.columns)

['CHR',
 'BP',
 'SNP',
 'CM',
 'base',
 'Coding_UCSC',
 'Coding_UCSC.flanking.500',
 'Conserved_LindbladToh',
 'Conserved_LindbladToh.flanking.500',
 'CTCF_Hoffman',
 'CTCF_Hoffman.flanking.500',
 'DGF_ENCODE',
 'DGF_ENCODE.flanking.500',
 'DHS_peaks_Trynka',
 'DHS_Trynka',
 'DHS_Trynka.flanking.500',
 'Enhancer_Andersson',
 'Enhancer_Andersson.flanking.500',
 'Enhancer_Hoffman',
 'Enhancer_Hoffman.flanking.500',
 'FetalDHS_Trynka',
 'FetalDHS_Trynka.flanking.500',
 'H3K27ac_Hnisz',
 'H3K27ac_Hnisz.flanking.500',
 'H3K27ac_PGC2',
 'H3K27ac_PGC2.flanking.500',
 'H3K4me1_peaks_Trynka',
 'H3K4me1_Trynka',
 'H3K4me1_Trynka.flanking.500',
 'H3K4me3_peaks_Trynka',
 'H3K4me3_Trynka',
 'H3K4me3_Trynka.flanking.500',
 'H3K9ac_peaks_Trynka',
 'H3K9ac_Trynka',
 'H3K9ac_Trynka.flanking.500',
 'Intron_UCSC',
 'Intron_UCSC.flanking.500',
 'PromoterFlanking_Hoffman',
 'PromoterFlanking_Hoffman.flanking.500',
 'Promoter_UCSC',
 'Promoter_UCSC.flanking.500',
 'Repressed_Hoffman',
 'Repressed_Hoffman.fl

In [43]:
cols_to_keep=['CHR',
 'BP',
 'SNP',
 'CM',
 'base',
 'Coding_UCSC',
 'Coding_UCSC.flanking.500',
 'Conserved_LindbladToh',
 'Conserved_LindbladToh.flanking.500',
 'CTCF_Hoffman',
 'CTCF_Hoffman.flanking.500',
 'DHS_peaks_Trynka',
 'DHS_Trynka',
 'DHS_Trynka.flanking.500',
 'Enhancer_Hoffman',
 'Enhancer_Hoffman.flanking.500','H3K27ac_Hnisz',
 'H3K27ac_Hnisz.flanking.500','H3K4me1_peaks_Trynka',
 'H3K4me1_Trynka',
 'H3K4me1_Trynka.flanking.500',
 'H3K4me3_peaks_Trynka',
 'H3K4me3_Trynka',
 'H3K4me3_Trynka.flanking.500',
 'H3K9ac_peaks_Trynka',
 'H3K9ac_Trynka',
 'H3K9ac_Trynka.flanking.500',
 'Intron_UCSC',
 'Intron_UCSC.flanking.500',
 'Repressed_Hoffman',
 'Repressed_Hoffman.flanking.500',
 'SuperEnhancer_Hnisz',
 'SuperEnhancer_Hnisz.flanking.500',
 'TFBS_ENCODE',
 'TFBS_ENCODE.flanking.500',
 'WeakEnhancer_Hoffman',
 'WeakEnhancer_Hoffman.flanking.500',
 'Conserved_Mammal_phastCons46way',
 'Conserved_Mammal_phastCons46way.flanking.500','BivFlnk',
 'BivFlnk.flanking.500'
 ]
#len(cols_to_keep)
#for x in cols_to_keep:
#    print(list(Full_baseline.columns).index(x)+1)
Subset_baseline_annot=pd.read_csv("/Users/xg1/Downloads/ldsc/subset36_baselineLD/baselineLD.1.annot.gz", sep='\t')
Subset_baseline_annot

,CHR,BP,SNP,CM,base,Coding_UCSC,Coding_UCSC.flanking.500,Conserved_LindbladToh,Conserved_LindbladToh.flanking.500,CTCF_Hoffman,...,SuperEnhancer_Hnisz,SuperEnhancer_Hnisz.flanking.500,TFBS_ENCODE,TFBS_ENCODE.flanking.500,WeakEnhancer_Hoffman,WeakEnhancer_Hoffman.flanking.500,Conserved_Mammal_phastCons46way,Conserved_Mammal_phastCons46way.flanking.500,BivFlnk,BivFlnk.flanking.500
0,1,11008,rs575272151,0.000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,11012,rs544419019,0.000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,13110,rs540538026,0.000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,13116,rs62635286,0.000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,1,13118,rs200579949,0.000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779349,1,249233164,rs374472180,293.397,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
779350,1,249236809,rs548489155,293.397,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
779351,1,249236810,rs569532168,293.397,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
779352,1,249239303,rs573070055,293.397,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [30]:
Subset_baseline=pd.read_csv("/Users/xg1/Downloads/ldsc/subset36_baselineLD/baselineLD.22.l2.ldscore.gz", sep='\t')
Subset_baseline

,CHR,SNP,BP,baseL2,Coding_UCSCL2,Coding_UCSC.flanking.500L2,Conserved_LindbladTohL2,Conserved_LindbladToh.flanking.500L2,CTCF_HoffmanL2,CTCF_Hoffman.flanking.500L2,...,SuperEnhancer_HniszL2,SuperEnhancer_Hnisz.flanking.500L2,TFBS_ENCODEL2,TFBS_ENCODE.flanking.500L2,WeakEnhancer_HoffmanL2,WeakEnhancer_Hoffman.flanking.500L2,Conserved_Mammal_phastCons46wayL2,Conserved_Mammal_phastCons46way.flanking.500L2,BivFlnkL2,BivFlnk.flanking.500L2
0,22,rs9617528,16061016,7.336,0.000,0.000,0.000,0.049,0.017,1.132,...,0.000,0.000,-0.001,-0.006,0.000,-0.004,0.004,0.075,0.000,0.000
1,22,rs4911642,16504399,62.101,0.951,0.112,0.035,0.908,0.330,1.274,...,0.140,0.001,0.657,1.294,0.005,0.161,1.082,1.519,0.082,0.124
2,22,rs7287144,16886873,152.659,0.040,0.087,0.219,7.742,3.154,14.665,...,0.209,0.029,6.047,11.638,0.076,1.920,0.589,14.653,0.374,1.094
3,22,rs5748662,16892858,130.432,0.041,0.081,0.183,6.689,2.628,12.756,...,0.116,0.018,5.004,9.752,0.075,1.721,0.533,12.682,0.310,0.958
4,22,rs5994034,16894090,50.683,0.025,0.219,0.043,3.836,1.940,4.231,...,0.527,0.056,2.785,2.740,0.131,0.165,0.280,6.042,0.214,0.404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17484,22,rs13056621,51181759,21.202,1.130,3.962,0.613,4.707,1.425,1.952,...,8.282,0.047,4.355,4.270,0.277,2.048,0.607,5.596,1.720,2.486
17485,22,rs3865766,51186228,53.753,2.574,8.992,0.668,11.365,1.582,5.186,...,12.061,0.097,6.609,8.605,0.463,4.204,0.877,11.767,2.083,3.756
17486,22,rs3888396,51211392,25.534,1.082,4.624,0.490,5.231,0.688,3.069,...,4.021,0.041,3.137,4.154,1.421,2.207,0.384,5.301,1.079,1.474
17487,22,rs2238837,51212875,42.265,2.677,6.184,0.437,8.493,0.936,3.294,...,9.023,0.045,4.561,6.306,0.280,3.048,0.655,8.143,1.526,2.506


In [29]:
Full_baseline_L2=pd.read_csv("/Users/xg1/Downloads/ldsc/baselineLD/baselineLD.22.l2.ldscore.gz", sep='\t')
cols_to_keep=['CHR',
 'BP',
 'SNP',
 'baseL2',
 'Coding_UCSCL2',
 'Coding_UCSC.flanking.500L2',
 'Conserved_LindbladTohL2',
 'Conserved_LindbladToh.flanking.500L2',
 'CTCF_HoffmanL2',
 'CTCF_Hoffman.flanking.500L2',
 'DHS_peaks_TrynkaL2',
 'DHS_TrynkaL2',
 'DHS_Trynka.flanking.500L2',
 'Enhancer_HoffmanL2',
 'Enhancer_Hoffman.flanking.500L2','H3K27ac_HniszL2',
 'H3K27ac_Hnisz.flanking.500L2','H3K4me1_peaks_TrynkaL2',
 'H3K4me1_TrynkaL2',
 'H3K4me1_Trynka.flanking.500L2',
 'H3K4me3_peaks_TrynkaL2',
 'H3K4me3_TrynkaL2',
 'H3K4me3_Trynka.flanking.500L2',
 'H3K9ac_peaks_TrynkaL2',
 'H3K9ac_TrynkaL2',
 'H3K9ac_Trynka.flanking.500L2',
 'Intron_UCSCL2',
 'Intron_UCSC.flanking.500L2',
 'Repressed_HoffmanL2',
 'Repressed_Hoffman.flanking.500L2',
 'SuperEnhancer_HniszL2',
 'SuperEnhancer_Hnisz.flanking.500L2',
 'TFBS_ENCODEL2',
 'TFBS_ENCODE.flanking.500L2',
 'WeakEnhancer_HoffmanL2',
 'WeakEnhancer_Hoffman.flanking.500L2',
 'Conserved_Mammal_phastCons46wayL2',
 'Conserved_Mammal_phastCons46way.flanking.500L2','BivFlnkL2',
 'BivFlnk.flanking.500L2'
 ]
for x in cols_to_keep:
    print(list(Full_baseline_L2.columns).index(x)+1)

1
3
2
4
5
6
7
8
9
10
13
14
15
18
19
22
23
26
27
28
29
30
31
32
33
34
35
36
41
42
43
44
45
46
55
56
84
85
88
89


In [17]:
Full_baseline_M2=pd.read_csv("/Users/xg1/Downloads/ldsc/baselineLD/baselineLD.22.l2.M_5_50", sep='\t')
#list(Full_baseline_M2.columns)
Full_baseline_M2


,83323.0,2185.0,7115.0,1800.0,25826.0,3165.0,6538.0,15260.0,39498.0,11036.0,...,633.0,4428.0,1563.0,642.0,849.0,537.0,1151.0,7881.0,357.0,83.0


In [18]:
for x in cols_to_keep:
    print(list(Full_baseline.columns).index(x)-3)

-3
-2
-1
0
1
2
3
4
5
6
7
10
11
12
15
16
19
20
23
24
25
26
27
28
29
30
31
32
33
38
39
40
41
42
43
52
53
81
82
85
86


In [32]:
subset_M=pd.read_csv("/Users/xg1/Downloads/ldsc/subset36_baselineLD/baselineLD.22.l2.M_5_50", sep='\t')
subset_M

,83323.0,2185.0,7115.0,1800.0,25826.0,3165.0,6538.0,11036.0,16126.0,31071.0,...,25177.0,504.0,17205.0,21718.0,3160.0,9414.0,1356.0,25054.0,2176.0,3483.0
